In [8]:
from scrollguard.utils import get_config
from scrollguard.etl import extract_csv, extract_xml 
import requests
import ssl
import urllib3

In [9]:
class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount("https://", CustomHttpAdapter(ctx))
    return session

In [11]:
# response = requests.get("https://www.treasury.gov/ofac/downloads/sdn.csv")
# response = requests.get("https://scsanctions.un.org/resources/xml/en/consolidated.xml")
response = get_legacy_session().get("https://scsanctions.un.org/resources/xml/en/consolidated.xml")

AttributeError: 'function' object has no attribute 'get'

In [8]:
source = get_config()["SOURCES"]["OFAC_SDN"]
source_format = source["FORMAT"]

if source_format=="CSV":
    data = extract_csv(source)
elif source_format=="XML":
    data = extract_xml(source)